In [3]:
# import topopy
from test_functions import *
# import matplotlib.pyplot as plt
# import samplers
# import seaborn as sns
# from matplotlib import patches, collections
import numpy as np
from ipywidgets import interact, HTML
import random
import pdir



In [4]:
from sidepanel import SidePanel

import ipywidgets as widgets

from regulus.utils import io
from regulus.topo import * 
from regulus.alg import *
from regulus.measures import *
from regulus.models import *
from regulus.tree import *

from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget
from ipyregulus.filters.filters import * 
from ipyregulus.filters.monitor import Monitor
from ipyregulus.tree.has_tree import *
from ipyregulus.alg.view import *


from regulus.topo import morse_smale, Regulus
from regulus.data.data import Data
from regulus.measures import *
from regulus.models import *
import pandas as pd

In [5]:
test_function = gerber_different_heights
samples = 200
X = np.random.rand(samples,2)
Y = test_function(X)

# Y = (Y - np.min(Y)) / (np.max(Y) - np.min(Y)) # Not sure whether this is needed 

all_pts = np.hstack((X,np.expand_dims(Y, axis=1)))

# X = samplers.CVTSampler.generate_samples(samples, 2, 1)

# msc = topopy.MorseSmaleComplex(graph="beta skeleton", gradient="steepest", max_neighbors=8, beta=1.0)
# msc.build(X, Y)


# NEW PART IN REG to create pts at runtime 

In [1]:
from Hartmann import create, calc_Hartmanny

In [34]:
all_pts = create(500)

In [22]:
regulus, df = create_reg(all_pts)
tree, data = process(regulus)

E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  return getattr(obj, method)(*args, **kwds)
E:\Anaconda\envs\CyPlot\lib\site-packages\numpy\core\fromnumeric.py:51: FutureWarning: 'argmax' is deprecated, use 'idxmax' instead. The behavior of 'argmax'
will be corrected to return the positional maximum in the future.
Use 'series.values.argmax' to get the position of the maximum now.
  return getattr(obj, method)(*args, **kwds)


found 10 singles
len(idx)= 258


In [21]:
all_pts = regenerate(all_pts, tree, data, test_function, df, 10)

In [23]:
treev = draw_tree(tree)
details = draw_details(data)
link_views(treev, details)
#widgets.link((treev.view, 'details'), (details, 'show'))

TreeView(field='span', tree_model=TreeWidget(attrs={'span': {0: 0.39232641952006786, 1: 0.22021486845974464, 2…

In [7]:

def create_reg(all_pts):
    cols = ['x'+str(i) for i in range(all_pts.shape[1])]

    cols[-1] = 'y'

    df = pd.DataFrame(all_pts, columns=cols)

    pts = Data.from_pts(df)

    # Normalize here  
    pts.normalize()

    regulus = morse_smale(pts, knn=8)#**kwargs)

    regulus.add_attr('linear', linear_model)
    regulus.add_attr('fitness', fitness)
    regulus.add_attr('relative_fitness', relative_fitness)

    regulus.tree.add_attr('parent_fitness', parent_fitness)
    regulus.tree.add_attr('child_fitness', child_fitness)
    regulus.tree.add_attr('size', node_size)
    regulus.tree.add_attr('rel_size', node_relative_size)
    regulus.tree.add_attr('span', node_span)
    return regulus, df




In [9]:
def process(regulus):
    from ipyregulus import TreeWidget, TreeView, DetailsView, DataWidget

    tree = TreeWidget(regulus.tree)
    data = DataWidget(data=regulus)
    return tree, data

# Use compute paraent similarity in the tree structure, stop at the level where value is below threshold 

In [20]:
import old_code.refine_topo_ as rt
from old_code.resample_ import get_pts, Resample

# new_x = np.vstack((X,X2))
def regenerate(all_pts, tree, data, test_function,df, step):
    dims = all_pts.shape[1]-1
    new_x = np.array([], dtype=np.float64).reshape(0,dims)

    leafnodes = rt.check_fit(0.4, tree=tree, reg_data = data, raw_data=all_pts)


    for i in leafnodes:
        ind = i.id
        ind = get_pts(data.partitions[ind],data.pts_loc)
        cur_p = df.loc[ind,:]
        r = Resample(cur_p)
        r.add_samples(step)
        out = r.get_input()
        new_x = np.vstack((new_x, out))
        # May need to rm duplicates 
    #print(new_x.shape)    
    new_y = test_function(new_x).reshape(-1,1)
    #print(new_y.shape)
    new_pts = np.hstack((new_x,new_y))

    all_pts2 = np.vstack((all_pts, new_pts))
    return all_pts2


# TreeView

In [12]:

def draw_tree(tree):
    treev = show_tree(tree)
    w = widgets.Dropdown(
        options=['span', 'fitness', 'parent_fitness', 'child_fitness'],
        value='span',
        description='Attribute:',
        disabled=False,
    )
    panel = SidePanel(title='Fitness')#, side='split-bottom', ref='SidePanel')
    with panel:
        display(w, treev.view, treev.filter)
    return treev
#widgets.link((w, 'value'), (v2.view, 'field'))

# Details View

In [13]:
def draw_details(data):
    details = DetailsView(data=data)

    p7 = SidePanel(title='details')
    with p7:
        display(details)
    return details

# Linking Two Views

In [14]:
def link_views(treev, details):
    widgets.link((treev.view, 'details'), (details, 'show'))


In [43]:
details.show = list(map(lambda x: x.id, leafnodes))
#details.show(ids)
#map(lambda x: x**2, items)
#map(function_to_apply, list_of_inputs)

In [37]:
out

array([[0.58544045, 0.08079815],
       [0.58629714, 0.07967075],
       [0.58421014, 0.08164452],
       [0.58421471, 0.08164199],
       [0.58402994, 0.08174176]])

# Create resample class, resampling a user specified partition, for User-guided resampling

In [28]:
from resample import get_pts, Resample

# Change the following line to use differennt partition to resample
ind = get_pts(data.partitions[0],data.pts_loc)

selectP = df.loc[ind,:]
r = Resample(selectP,pred_method="bounding_box")

# A line chart for a specified partition, different interactions could be added 

In [1]:
from cyplot import plot
myp = plot(selectP,index = 'y',debug=True, dims=['x0','x1','y'])#ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp.enable(['brush_x','brush'])
myp.on('brush_x', r.cb)
myp.on('brush', r.cb)

display(myp)

NameError: name 'selectP' is not defined

In [30]:
# from cyplot import plot
myp2 = plot(selectP,index = 'y',debug=True, ptype='PCA')#, ptype='line'/ PCA. line. dims=[]
myp2.enable(['brush_x','brush'])
myp2.on('brush_x', r.cb)
myp2.on('brush', r.cb)
display(myp2)

In [31]:
myp2.enable('x_brush')
display(myp2)

# random generate pts based on user selected range of function value

In [32]:
r.add_samples(5) # user's specified # of samples


# When user adds enough samples, they could use regression to predict the point locations to resample 

In [33]:
# Would scale back if needed 
new_x = r.get_input()
new_y = test_function(new_x).reshape(-1,1)
new_pts = np.hstack((new_x,new_y))



In [34]:
all_pts = np.vstack((all_pts, new_pts))